# 7. Zbudować (samodzielnie!) model predykcji (regresji) dla danych rzeczywistych pochodzących z internetu np. ze strony https://archive.ics.uci.edu/ml/datasets lub https://www.kaggle.com/datasets Proszę pamiętać o wykonaniu następujących kroków: załadowanie danych, wizualizacja, przygotowanie danych (nulls, konwersja do liczb, one-hot-encoding, normalizacja lub standaryzacja),  stworzenie modelu, ocena jakości modelu (różne wskaźniki: MAE, MSE, MAPE), eksperymenty z różnymi parametrami.

### 1. Przygotowanie danych do uczenia

In [168]:
import kagglehub
import pandas as pd
import torch

from sklearn.model_selection import train_test_split

In [169]:
# pobranie pliku z pingwinami z kaggla i utworzenie ścieżki do pliku csv

path = kagglehub.dataset_download("amulyas/penguin-size-dataset")
path_to_file = path + "/" + "penguins_size.csv"
path_to_file

'/home/radekbys/.cache/kagglehub/datasets/amulyas/penguin-size-dataset/versions/1/penguins_size.csv'

In [170]:
df = pd.read_csv(path_to_file)                  # wczytanie pliku z csv
df = df.dropna()                                # odrzucenie błędnych wartości ze zbioru danych
df= df[df["sex"] != "."]                        # odrzucić wartości gdzie płeć jest nieznana
df = pd.get_dummies(df)                         # zakodowanie lokacji, gatunku, płci do one hot encoding
df = df.astype("float32")                       # konwersja wszysdtkich danych do float 32
body_mass = df["body_mass_g"]
df = (df - df.min()) / (df.max() - df.min())    # normalizacja wszystkich danych do zakresu między 0 a 1 
df["body_mass_g"] = body_mass
df

,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_FEMALE,sex_MALE
0,0.254545,0.666667,0.152542,3750.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.269091,0.511905,0.237288,3800.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.298182,0.583333,0.389830,3250.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.167273,0.738095,0.355932,3450.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
5,0.261818,0.892857,0.305085,3650.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
338,0.549091,0.071429,0.711864,4925.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
340,0.534545,0.142857,0.728814,4850.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
341,0.665455,0.309524,0.847458,5750.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
342,0.476364,0.202381,0.677966,5200.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [171]:
y_np = df["body_mass_g"].values
df.drop("body_mass_g", axis=1)
X_np = df.values

y_np.shape, X_np.shape

((333,), (333, 12))

In [172]:

X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.15, random_state=42)

X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train).unsqueeze(dim=1)
y_test = torch.from_numpy(y_test).unsqueeze(dim=1)


X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_train.dtype

(torch.Size([283, 12]),
 torch.Size([50, 12]),
 torch.Size([283, 1]),
 torch.Size([50, 1]),
 torch.float32)

### 2. Przygotowanie modelu

In [173]:
class PenguinWeigthFindingModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(in_features=12, out_features= 128),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(in_features=128, out_features= 1)
        )
        
    def forward(self, x):
        return self.layers(x)
    
model = PenguinWeigthFindingModel()
model

PenguinWeigthFindingModel(
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [174]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
lossfn = torch.nn.L1Loss()

### 3.0 Uczenie i testowanie

In [175]:
epochs = 2001

for epoch in range(epochs):
    model.train()
    logits = model(X_train)
    loss = lossfn(logits, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        model.eval()
        with torch.inference_mode():
            test_logits = model(X_test)
            test_loss = lossfn(test_logits,  y_test)
            print(F"epoch: {epoch} | mean absolute error: {test_loss.item():.2f}g")
            
    

epoch: 0 | mean absolute error: 4422.62g
epoch: 100 | mean absolute error: 3934.32g
epoch: 200 | mean absolute error: 3428.91g
epoch: 300 | mean absolute error: 2881.35g
epoch: 400 | mean absolute error: 2277.05g
epoch: 500 | mean absolute error: 1608.87g
epoch: 600 | mean absolute error: 859.75g
epoch: 700 | mean absolute error: 19.50g
epoch: 800 | mean absolute error: 0.36g
epoch: 900 | mean absolute error: 0.26g
epoch: 1000 | mean absolute error: 0.21g
epoch: 1100 | mean absolute error: 0.18g
epoch: 1200 | mean absolute error: 0.93g
epoch: 1300 | mean absolute error: 1.47g
epoch: 1400 | mean absolute error: 0.41g
epoch: 1500 | mean absolute error: 0.62g
epoch: 1600 | mean absolute error: 1.26g
epoch: 1700 | mean absolute error: 0.41g
epoch: 1800 | mean absolute error: 0.59g
epoch: 1900 | mean absolute error: 1.17g
epoch: 2000 | mean absolute error: 0.39g


In [180]:
model.eval()
model(X_test[:10]).detach(), y_test[:10]

(tensor([[3250.4055],
         [4875.4595],
         [4000.3721],
         [3675.3728],
         [4050.4458],
         [4750.3618],
         [5550.2310],
         [3600.3706],
         [3775.2727],
         [3950.3342]]),
 tensor([[3250.],
         [4875.],
         [4000.],
         [3675.],
         [4050.],
         [4750.],
         [5550.],
         [3600.],
         [3775.],
         [3950.]]))